In [8]:
using StatsBase
using Combinatorics
include("jl/vol.jl");

In [9]:
n = 50000

Z = rand(1:50, n)
D = rand(2:100, n)

r = 10 # maximum hyperedge size

V, μ, M = evalSums(Z, D, r;constants=true, bigInt=true);

In [10]:
s = 4
t = 5
to_move = findall(==(s), Z);
println("We are going to move $(length(to_move)) nodes from group $s to group $t")

We are going to move 985 nodes from group 4 to group 5


In [11]:
function naive(Z, V, μ, M)
    """
    naive version loops through one-node moves
    """
    Z_ = copy(Z)
    for i in to_move
        ΔV, Δμ, ΔM = increments(V, μ, M, i, t, D, Z_)
        V, μ, M = addIncrements(V, μ, M, ΔV, Δμ, ΔM)
        # carry out the change in membership
        Z_[i] = t
    end
    return(V, μ, M)
end

function savvy(Z, V, μ, M)
    """
    savvy version computes the increments associated with moving all nodes as a batch. Uses a different method of increments
    """
    Z_ = copy(Z)
    ΔV, Δμ, ΔM = increments(V, μ, M, to_move, t, D, Z)
    V, μ, M = addIncrements(V, μ, M, ΔV, Δμ, ΔM)
    return(V, μ, M)
end

savvy (generic function with 1 method)

In [14]:
@time M_n = naive(Z, V, μ, M)[3]

  0.598837 seconds (5.80 M allocations: 382.755 MiB, 14.23% gc time)


In [15]:
@time M_s = savvy(Z, V, μ, M)[3]

  0.001031 seconds (11.78 k allocations: 881.250 KiB)


In [17]:
all([M_s[p] == M_n[p] for p in keys(M_n)]) # check equality

true